In [3]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
with open('roadapikey.txt', mode='r') as key_file:
    road_key = key_file.read()

In [5]:
# 데이터 로드
park = pd.read_csv('서울시 주요 공원현황.csv', encoding='euc-kr')
park.shape, park.head(2)

((132, 20),
    공원번호       공원명                                               공원개요  \
 0     1  남산도시자연공원  남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...   
 1     2    길동생태공원  길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...   
 
                                              면적        개원일  \
 0  2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡  1968.9.10   
 1                                        80683㎡  1999.5.20   
 
                                                 주요시설  \
 0  기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...   
 1               탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터   
 
                                         주요식물  \
 0          소나무 단풍 아카시아 상수리나무 등 191종 2881870주   
 1  소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본   
 
                                                  안내도  \
 0  http://parks.seoul.go.kr/upload/seditorMulti/2...   
 1  http://parks.seoul.go.kr/template/common/img/p...   
 
                                                 오시는길  \
 0  명동역(남산케이블카 와룡묘 서울애니메

In [6]:
# 불필요한 칼럼 제거
park.drop(columns=['공원번호', '공원개요', '개원일', '주요시설', '주요식물',
                '안내도', '오시는길', '이용시참고사항', '이미지', '지역', '관리부서', '전화번호',
                'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', '바로가기'], inplace=True,  axis=1,)
park.head(2)

,공원명,면적,공원주소,X좌표(WGS84),Y좌표(WGS84)
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394


In [7]:
# 칼럼 이름 변경
park.rename(columns={'X좌표(WGS84)':'경도'}, inplace=True)
park.rename(columns={'Y좌표(WGS84)':'위도'}, inplace=True)
park

,공원명,면적,공원주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),109635㎡,서울특별시 중랑구 면목7동 산50,127.095997,37.579762
128,문화비축기지,140022㎡,서울특별시 마포구 증산로 87,126.893246,37.571718
129,경춘선숲길,211392㎡,노원구 공릉2동 산 82-2,127.076482,37.627077
130,율현공원,157535㎡,서울특별시 강남구 율현동 밤고개로24길(율현동 56),127.115594,37.472332


In [8]:
# 널 값 제거
park=park.dropna(axis=0)
park

,공원명,면적,공원주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
...,...,...,...,...,...
126,서일대뒷산공원,631021㎡,서울특별시 중랑구 면목동 산19,127.098782,37.587716
127,용마도시자연공원(사가정공원),109635㎡,서울특별시 중랑구 면목7동 산50,127.095997,37.579762
128,문화비축기지,140022㎡,서울특별시 마포구 증산로 87,126.893246,37.571718
129,경춘선숲길,211392㎡,노원구 공릉2동 산 82-2,127.076482,37.627077


In [9]:
# addr -> str 으로 변환
addr = park['공원주소'].str.replace(',','')[0]
addr

'서울특별시 중구 삼일대로 231(예장동)'

In [10]:
park.to_csv('addr_non.csv', index=False, sep=',',encoding='utf8')

In [11]:
df_non = pd.read_csv('addr_non.csv')
df_non

,공원명,면적,공원주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805
...,...,...,...,...,...
124,서일대뒷산공원,631021㎡,서울특별시 중랑구 면목동 산19,127.098782,37.587716
125,용마도시자연공원(사가정공원),109635㎡,서울특별시 중랑구 면목7동 산50,127.095997,37.579762
126,문화비축기지,140022㎡,서울특별시 마포구 증산로 87,126.893246,37.571718
127,경춘선숲길,211392㎡,노원구 공릉2동 산 82-2,127.076482,37.627077


In [19]:
df_non.columns

Index(['공원명', '면적', '공원주소', '경도', '위도'], dtype='object')

In [16]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

In [23]:
import folium
import matplotlib as mpl

map = folium.Map(location=[df_non['위도'].mean(), df_non['경도'].mean()], zoom_start =10)

for i in df_non.index:
    park_name = df_non['공원명'] + ' - ' + df_non['공원주소']
    icon_color = 'red'
    folium.features.CircleMarker(
        location=[df_non['위도'][i],df_non['경도'][i]], 
                        radius=df_non['면적'],
                        popup=park_name,
                        color= icon_color,
                        fill=True,
                        fill_color=icon_color
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시 공원면적 비교</b></h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().